In [11]:
import neptune
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import os
import json
from Models.AutoEncoder import AutoEncoder, AE_validDataset, AE_trainDataset
from utils.utils import process_data
from sklearn.metrics import f1_score
import numpy as np

# Auto Encoder Test

* L1 정규화 추가
* weight_decay 추가
* BatchNorm 추가

## Base Setup

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [13]:
num_epochs = 100
batch_size = 128
lr = 1e-4
encoding_dim = 28

In [14]:
# Feature Selection
cat_features = ['Card', 'Gender', 'Card Brand', 'Card Type', 'Expires', 'Has Chip', 
                'Year PIN last Changed', 'Whether Security Chip is Used', 'Day']
num_features = ['Current Age', 'Retirement Age', 'Per Capita Income - Zipcode', 'Zipcode',
                'Yearly Income', 'Total Debt', 'Credit Score', 'Credit Limit', 'Amount']
discarded = ['User', 'Birth Year', 'Birth Month']
print(len(cat_features)*5 + len(num_features))


54


In [15]:
# 데이터 전처리
data_path = 'Data/[24-2 DS_Project2] Data.csv'
(train_cat_X, train_num_X, train_y), (valid_cat_X, valid_num_X, valid_y), label_encoders = process_data(
    data_path,
    cat_features,
    num_features,
    discarded
)

TRANSITION
IQR
SPLIT
DISCARD
SCALE
ENCODE
UNLABEL
TARGET
TRAIN CAT/NUM
VALID CAT/NUM
RETURN


In [16]:
print(train_y.shape)
print(train_cat_X.shape)

(902733, 1)
(902733, 9)


In [17]:
train_cat_X,
train_num_X,
train_y,
valid_cat_X,
valid_num_X,
valid_y,
label_encoders

{'Card': LabelEncoder(),
 'Gender': LabelEncoder(),
 'Card Brand': LabelEncoder(),
 'Card Type': LabelEncoder(),
 'Expires': LabelEncoder(),
 'Has Chip': LabelEncoder(),
 'Year PIN last Changed': LabelEncoder(),
 'Whether Security Chip is Used': LabelEncoder(),
 'Day': LabelEncoder()}

In [18]:
train_dataset = AE_trainDataset(train_cat_X, train_num_X, device)
valid_dataset = AE_validDataset(valid_cat_X, valid_num_X, valid_y, device)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

In [19]:
model = AutoEncoder(encoding_dim=encoding_dim, cat_features=cat_features, num_features=num_features).to(device)
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
criterion = nn.MSELoss()


In [21]:
# Kaiming Initialization 적용
for name, param in model.named_parameters():
    if isinstance(param, nn.Linear):
        nn.init.kaiming_normal_(param.weight, mode='fan_in', nonlinearity='relu')
        if param.bias is not None:
            nn.init.zeros_(param.bias)

In [22]:
best_f1 = 0
l1_lambda = 1e-5
from tqdm import tqdm
for epoch in tqdm(range(num_epochs)):
    # 학습 단계
    model.train()
    train_loss = 0
    for cat_features, num_features in train_loader:
        optimizer.zero_grad()
        y_hat, y = model(cat_features, num_features)
        
        # MSE 손실 계산
        mse_loss = criterion(y_hat, y)
        
        # L1 정규화 계산
        l1_reg = torch.tensor(0., requires_grad=True).to(device)
        for param in model.parameters():
            l1_reg = l1_reg + torch.norm(param, 1)
        
        # 총 손실 = MSE 손실 + L1 정규화
        loss = mse_loss + l1_lambda * l1_reg
        
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    # 평균 train_loss 계산
    train_loss /= len(train_loader)
    
    # 검증 단계 (10 에포크마다)
    if epoch % 10 == 0:
        model.eval()
        valid_loss = 0
        reconstruction_errors = []
        all_labels = []
        
        with torch.no_grad():
            for cat_features, num_features, labels in valid_loader:
                y_hat, y = model(cat_features, num_features)
                batch_loss = criterion(y_hat, y)
                valid_loss += batch_loss.item()
                
                sample_errors = torch.mean((y_hat - y) ** 2, dim=1)
                reconstruction_errors.extend(sample_errors.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
            
            # 성능 평가
            threshold = np.percentile(reconstruction_errors, 90)
            predictions = (np.array(reconstruction_errors) > threshold).astype(int)
            f1 = f1_score(all_labels, predictions)
            
            # 결과 로깅
            print(f"Epoch {epoch}: Valid Loss = {valid_loss:.4f}, F1 Score = {f1:.4f}")
            
            # 최고 성능 모델 저장
            if f1 > best_f1:
                best_f1 = f1

print(f'best F1 Score: {best_f1}')

  0%|          | 0/100 [00:47<?, ?it/s]


KeyboardInterrupt: 